# Analisi del consenso sui Bitcoin

Su richiesta di un'azienda di ricerche di mercato, si è svolta un'analisi del consenso sui Bitcoin a partire da un dataset di tweet sull'argomento. I risultati dell'analisi sono riportati in questo notebook, che ne segue le varie fasi dalla preparazione dei dati fino ai principali risultati.

Nella prima sezione saranno mostrate l'importazione dei dati forniti dall'azienda e la loro preparazione per l'analisi. Dopo una sezione dedicata a una panoramica sul dataset ottenuto, è stato analizzato il sentiment dal punto di vista temporale e dal punto di vista delle reazioni ai tweet.

## Importazione e preparazione dei dati

Questa sezione è dedicata all'importazione e alla preparazione dei dati per l'analisi.

Si inizierà con la preparazione del dataset relativo ai tweet, successivamente si elaboreranno i dati sui prezzi del bitcoin e in ultima analisi si uniranno i dataset ottenuti.

### Il dataset *bitcoin_tweets*

In questa prima sezione saranno importati e preparati i dati relativi ai tweet sui Bitcoin, forniti dall'azienda tramite un file CSV [a questo link](https://proai-datasets.s3.eu-west-3.amazonaws.com/bitcoin_tweets.csv).

Dopo aver scaricato il dataset, è stato stampato l'elenco delle colonne.

In [0]:
import pandas as pd

df = spark.createDataFrame(pd.read_csv("https://proai-datasets.s3.eu-west-3.amazonaws.com/bitcoin_tweets.csv",delimiter=","))
df.printSchema()

Delle colonne mostrate, si è deciso di mantenere le seguenti:
- La colonna *timestamp*, che riporta la data e l'ora del tweet;
- La colonna *text* contenente il testo del tweet;
- Le colonne *replies*, *likes* e *retweets* che contengono rispettivamente le risposte, i "mi piace" e le condivisioni ottenute dal tweet.

In [0]:
df=df.select("timestamp","text","replies","likes","retweets")
display(df)

Dalla colonna *timestamp* sono state calcolate le colonne *date* e *time*, che riportano la data e l'ora del tweet.

In [0]:
from pyspark.sql.functions import to_date, date_format

df=df.withColumn("date",to_date(df.timestamp))
df=df.withColumn("time",date_format(df.timestamp,"HH:mm"))
display(df)

Dalla colonna *time*, in particolare, è stata ottenuta la colonna *time_slot* contenente la fascia oraria di pubblicazione del tweet, partendo dal presupposto che l'orario indicato sia relativo al fuso orario di Washington D.C.

In [0]:
def get_time_slot(time):
    time_slot=int(time.split(":")[0])
    return f"{time_slot}-{time_slot+1}"

df=df.withColumn("time_slot",udf(get_time_slot)(df.time))
display(df)

Per quanto riguarda il sentiment, si è deciso di ricorrere a un modello già pronto per stimarlo dai testi dei tweet. Per fare questo è stata installata la libreria TextBlob, che esegue operazioni di Natural Language Processing.

In [0]:
!pip install textblob

In [0]:
%restart_python

Utilizzando la funzione *get_sentiment*, è stata calcolata la colonna *sentiment* che assegna a ogni valore della colonna *text* un sentimento positivo, negativo o neutro.

In [0]:
from textblob import TextBlob

def get_sentiment(text):
    result=TextBlob(text)
    if result.sentiment.polarity<0:
        return "negative"
    elif result.sentiment.polarity==0:
        return "neutral"
    else:
        return "positive"
    
df=df.withColumn("sentiment",udf(get_sentiment)(df.text))
display(df)

Infine, sommando i valori nelle colonne *likes*, *replies* e *retweets*, è stata ottenuta la colonna *reactions* che conta le reazioni totali al tweet.

In [0]:
df=df.withColumn("reactions",df.likes+df.replies+df.retweets)
display(df)

Il dataset ottenuto finora è stato salvato come tabella in modo da poter essere ripreso nell'ultima parte di questa 
sezione.

In [0]:
df.write.mode("overwrite").option("overwriteSchema","true").saveAsTable("default.bitcoin_tweets")

### Il dataset *btc_usd*

In questa sottosezione si prepareranno i dati relativi allo storico dei prezzi dei Bitcoin, ottenuti tramite file CSV scaricato [da questo link](https://finance.yahoo.com/quote/BTC-USD/history/).

Dopo aver importato i dati nel Catalogo di Databricks, la tabella è stata importata tramite una query SQL ed è stato stampato l'elenco delle colonne.

In [0]:
btc_df=spark.sql("select date, open_price, close_price from default.btc_usd")
btc_df.printSchema()

Nella tabella importata in questo notebook troviamo le seguenti colonne:
- La colonna *date* riporta la data dell'osservazione sui prezzi;
- La colonna *open_price* indica il prezzo all'apertura delle borse;
- La colonna *close_price* indica il prezzo alla chiusura delle borse.

Dalle ultime due colonne è stata calcolata la colonna *mean_price*, che indica il prezzo medio dei Bitcoin. Per semplicità, si è posto che il prezzo medio sia dato dalla media aritmetica dei prezzi di apertura e chiusura.

In [0]:
btc_df=btc_df.withColumn("mean_price",(btc_df.open_price+btc_df.close_price)/2)
display(btc_df)

Per facilitare le operazioni nella parte finale di questa sezione, la tabella appena ottenuta è stata salvata come *bitcoin_usd*.

In [0]:
btc_df.write.mode("overwrite").option("overwriteSchema","true").saveAsTable("default.bitcoin_usd")

### Il dataset *project_data*

In questa ultima sottosezione sono state unite le tabelle ottenute in precedenza e sono stati filtrati i dati su cui si svolgerà l'analisi.

Per iniziare, tramite una query SQL, è stata effettuata una left join sulle tabelle *bitcoin_tweets* e *bitcoin_usd* e sono stati mostrati i soli record dove è presente il valore del prezzo medio.

In [0]:
df=spark.sql("""
             select tweets.*, usd.open_price, usd.close_price, usd.mean_price
             from default.bitcoin_tweets tweets
             left join default.bitcoin_usd usd
             on tweets.`date`=usd.`date`
             where usd.mean_price is not null
             """)
df.printSchema()

Delle colonne sopra riportate si è deciso di mantenere le seguenti:
- Le colonne *date* e *time_slot* con la data e la fascia oraria del tweet;
- La colonna *sentiment* con il sentimento positivo, negativo o neutro del tweet;
- Le colonne *likes*, *replies*, *retweets* e *reactions* con le reazioni al tweet;
- Le colonne *open_price*, *close_price* e *mean_price* con i prezzi di apertura, chiusura e medio nel giorno del tweet.

Tramite la funzione display, si è scaricata una copia della tabella tramite file CSV.

In [0]:
df=df.select("date","time_slot","sentiment","likes","replies","retweets","reactions","open_price","close_price","mean_price")
display(df)

Oltre al file CSV, i dati sono stati salvati nella tabella *project_data* nel Catalog di Databricks. Nelle prossime sezioni, le analisi saranno svolte a partire da questi dati.

In [0]:
df.write.mode("overwrite").option("overwriteSchema","true").saveAsTable("default.project_data")

## Panoramica generale

In questa sezione si offrirà una panoramica generale del dataset ottenuto nella sezione precedente e di seguito importato.

In [0]:
df=spark.sql("select * from default.project_data")
df.printSchema()

Di seguito si mostrerà il conteggio dei record che, ricordiamo, corrisponde al numero di tweet osservati.

In [0]:
df.count()

Il dataset è costituito da poco meno di un milione di tweet. Ricordiamo che il numero ridotto di record è dovuto al filtro dei dati ottenuto tramite la tabella *btc_usd*, in quanto si è deciso di mantenere solo i tweet per cui era presente il prezzo medio.

Il grafico di seguito ottenuto mostra l'andamento del prezzo medio nel tempo.

In [0]:
display(df.groupBy("date").mean("mean_price"))

Osserviamo una crescita esponenziale del prezzo del Bitcoin fino a dicembre 2017, seguito da un calo fino al minimo del prezzo raggiunto tra dicembre 2018 e marzo 2019.

In modo analogo, è stato tracciato un grafico che mostra il numero di tweet nel tempo. Per facilitare la lettura del grafico, si è deciso di utilizzare una scala logaritmica.

In [0]:
display(df.groupBy("date").count())

Si osserva che la crescita effettiva del numero di tweet è iniziata dopo il 2017 e che si è raggiunto un picco nel maggio 2019, in corrispondenza alla risalita del prezzo nel grafico a linee precedente. La relazione tra queste variabili sarà esplorata nella sezione dedicata alla panoramica temporale.

Il grafico a barre seguente mostra la media del numero di tweet per fascia oraria.

In [0]:
display(df.groupBy("date","time_slot").count().groupBy("time_slot").mean())

In media, il maggior numero di tweet viene pubblicato tra le 11 e 12, in corrispondenza della metà della giornata di operazioni in borsa, mentre il minore numero viene pubblicato tra le 19 e le 20, ossia dopo la chiusura dele borse.

Per quanto riguarda le reazioni ai tweet, di seguito si mostra una tabella in cui è stato calcolato il numero delle reazioni medie, che si aggira tra le 10 e le 11. Il dettaglio sulle reazioni ai tweet sarà esplorato nell'ultima sezione del notebook.

In [0]:
display(df.select("reactions").groupBy().mean())

Infine, mostriamo la composizione del dataset in base al sentiment dei tweet.

In [0]:
display(df.groupBy("sentiment").count())

Osserviamo che la maggioranza dei tweet ha un sentimento neutrale, probabilmente dovuto a una natura informativa o dimostrativa del tweet, mentre nel resto dei dati prevale un sentimento di tipo positivo.

## Panoramica storica

In questa sezione si analizzerà la relazione tra il sentiment e il prezzo in funzione del tempo. Si premette che, dato il diverso ordine di grandezza di queste variabili, si è scelto di utilizzare una scala logaritmica per agevolare la lettura del grafico.

Il grafico sottostante mostra l'andamento del numero di tweet (in blu) e del prezzo medio (in verde chiaro) in funzione dei mesi.

In [0]:
display(spark.sql("select date, count(*), mean(mean_price) from default.project_data group by `date`"))

Come già anticipato nella precedente sezione, il numero di tweet sul Bitcoin è cresciuto insieme al prezzo e ha continuato a salire anche in seguito al crollo tra gennaio e settembre 2018. Dopo quella data, i due grafici hanno ripreso ad andare quasi di pari passo.

La cella che segue mostra due grafici:
- Nella scheda "sentiment_per_time" si mostra l'andamento del sentiment in funzione del tempo;
- Nella scheda "price_sentiment_per_time" si mostrano invece il numero dei tweet positivi e negativi e il prezzo medio in funzione del tempo.

In [0]:
display(spark.sql("select date, sum(case when sentiment='positive' then 1.0 else 0.0 end) as n_positives, sum(case when sentiment='negative' then 1.0 else 0.0 end) as n_negatives, sum(case when sentiment='neutral' then 1.0 else 0.0 end) as n_neutrals, mean(mean_price) as mean_price from default.project_data group by date"))

Nel grafico "sentiment_per_time" si osserva una generica prevalenza di tweet neutrali, seguiti dai tweet positivi. Questa tendenza si è invertita nel febbraio 2018 e tra dicembre 2018 e aprile 2019, quando il prezzo medio ha iniziato a salire.

Per quanto riguarda il grafico "price_sentiment_per_time" si nota un andamento simile nel numero di tweet positivi e negativi, specialmente dopo il calo di prezzo medio nel primo semestre 2018.

Il grafico seguente mostra il sentiment medio dei tweet in base alla fascia oraria.

In [0]:
display(df.groupBy("date","time_slot","sentiment").count().groupBy("time_slot","sentiment").mean())

La tendenza generale è una prevalenza di tweet neutrali per la maggior parte della giornata. Tuttavia, mentre la tendenza delle date osserva una prevalenza di tweet neutrali e positivi, nelle fasce orarie si registra in media uno spostamento del sentiment tra il neutrale e il negativo.

Nei tre grafici seguenti, si mostra la suddivisione in fascia oraria in base al sentiment.

In [0]:
display(df.filter(df.sentiment=="neutral").groupBy("date","time_slot").count().groupBy("time_slot").mean())

In [0]:
display(df.filter(df.sentiment=="positive").groupBy("date","time_slot").count().groupBy("time_slot").mean())

In [0]:
display(df.filter(df.sentiment=="negative").groupBy("date","time_slot").count().groupBy("time_slot").mean())

Dai grafici appena mostrati possiamo dedurre che:
- Il picco di tweet neutrali viene osservato tra le 4 e le 5 del mattino, il che fa supporre che si trattino di tweet informativi che precedono l'apertura delle borse;
- Il massimo dei tweet negativi e positivi si registra intorno a metà giornata di operazioni in borsa, ossia tra le 11 e le 12 per i tweet positivi e tra le 12 e le 13 per i tweet negativi. Questo porta a supporre che si trattino di tweet "checkpoint" per fare il punto dell'andamento del prezzo a metà giornata;
- Il minimo dei tweet neutrali e negativi si registra nelle ore serali, comportamento che porta a ipotizzare che si trattino soprattutto di tweet "riepilogativi" sulla giornata appena trascorsa.

Per l'ultima parte della panoramica temporale, si è deciso di focalizzare l'attenzione sui tweet pubblicati intorno all'apertura delle borse (9:30 del mattino) e alla loro chiusura (16:00 del pomeriggio). Per facilitare l'estrazione delle informazioni, i dati filtrati sono stati salvati nelle tabelle *open_tab* e *close_tab*

In [0]:
df_open=df.filter((df.time_slot=="8-9") | (df.time_slot=="9-10")).select("date","sentiment","open_price")
df_close=df.filter((df.time_slot=="15-16") | (df.time_slot=="16-17")).select("date","sentiment","close_price")

In [0]:
df_open.write.mode("overwrite").option("overwriteSchema","true").saveAsTable("default.open_tab")
df_close.write.mode("overwrite").option("overwriteSchema","true").saveAsTable("default.close_tab")

I due grafici mostrano l'andamento del sentiment positivo e negativo e del prezzo medio in funzione del tempo sia all'apertura che alla chiusura.

In [0]:
display(spark.sql("""
                  select date,
                  mean(open_price) as open_price,
                  sum(case when sentiment="positive" then 1 else 0 end) as n_positives,
                  sum(case when sentiment="negative" then 1 else 0 end) as n_negatives,
                  sum(case when sentiment="neutral" then 1 else 0 end) as n_neutrals
                  from default.open_tab
                  group by 1
                  """))

In [0]:
display(spark.sql("""
                  select date,
                  mean(close_price) as close_price,
                  sum(case when sentiment="positive" then 1 else 0 end) as n_positives,
                  sum(case when sentiment="negative" then 1 else 0 end) as n_negatives,
                  sum(case when sentiment="neutral" then 1 else 0 end) as n_neutrals
                  from default.close_tab
                  group by 1
                  """))

Osserviamo che i due grafici sono molto simili nella forma, ma che il sentiment negativo è emerso di più nel corso dell'apertura delle borse rispetto alle fasce orarie intorno alla chiusura.

## Panoramica delle reazioni
In questa sezione sarà mostrata la relazione tra le reazioni ai tweet e il loro sentiment. In particolare, escluderemo da questa analisi i tweet neutrali per concentrarci sui tweet più polarizzanti.

Innanzitutto, dalla tabella *project_data* importeremo solo le colonne relative al sentiment, alle reazioni totali e al numero di reazioni per ciascun tipo.

In [0]:
df=spark.sql("select sentiment, likes, replies, retweets, reactions from default.project_data where sentiment<>'neutral'")
display(df)

Il grafico di seguito riportato mostra la media di reazioni per tweet in base al sentiment.

In [0]:
display(df.select("sentiment","reactions").groupBy("sentiment").mean())

Osserviamo che, in media, un tweet con sentiment negativo attira meno reazioni rispetto a un tweet positivo, anche se la differenza tra le due medie è minima.

Nella cella sottostante sono riportati tre grafici:
- La scheda "likes" mostra il numero medio di like in base al sentiment;
- La scheda "replies" mostra il numero medio di risposte in base al sentiment;
- La scheda "retweets" mostra il numero medio di condivisioni in base al sentiment.

In [0]:
display(df.select("sentiment","likes","replies","retweets").groupBy("sentiment").mean())

Dai grafici sopra mostrati emerge quanto segue:
- In media, un tweet negativo attira più reazioni di tipo passivo (come appunto i like), mentre un tweet positivo attira più reazioni che richiedono una maggiore attività da parte dell'utente (come possono essere le risposte e/o le condivisioni);
- Nelle reazioni attive, il divario tra tweet positivo e tweet negativo è maggiore quando si tratta di condivisioni rispetto alle risposte.

## Conclusioni

In questo notebook è stata analizzata una raccolta di tweet allo scopo di effettuare un analisi di consenso sul Bitcoin.

La prima sezione è stata ampiamente dedicata alla preparazione dei dati. Dopo aver importato e trasformato i dati relativi ai tweet e allo storico dei prezzi, i due dataset sono stati uniti in un'unica tabella successivamente utilizzata per l'analisi in oggetto.

La seconda sezione è dedicata a una panoramica generale dei dati ottenuti. Sono state infatti esplorate l'andamento dei prezzi e del numero di tweet, la media delle reazioni e la composizione del dataset in base al sentiment.

Nella terza sezione ci si è concentrati sulla panoramica storica e, in particolare, sulla relazione tra il prezzo del Bitcoin, il numero di tweet e il loro sentiment. L'analisi della panoramica storica si è svolta sia dal punto di vista della data che da quello della fascia oraria, con particolare enfasi sui dati negli orari di apertura e chiusura delle borse.

Infine, nella quarta sezione, è stato esplorato il rapporto tra sentiment e reazioni ai tweet, concentrandoci in particolare sui tweet positivi e negativi. Dopo una panoramica generale sulle reazioni totali, l'analisi si è concentrata su ciascun tipo di reazione (like, risposta e condivisione) in base al sentiment del tweet.